In [1]:
import pandas as pd

In [2]:
pd.read_pickle('./pred/Anaheim_GraphSAGE_confgnn.pkl')

{0: {'gnn': {'CQR': (np.float64(0.9127322404371584), np.float32(2.6241653))},
  'conf_gnn': {'CQR': (np.float64(0.9127322404371584), np.float32(2.6241653)),
   'eff_valid': np.float32(2.6870975)},
  'metrics': {'CWC': np.float64(1.0253191844460179),
   'PCIP': np.float64(0.9234972677595629),
   'NMPIW': np.float32(0.5726145),
   'MCT': np.float64(0.06392471451576942),
   'MPE': np.float32(0.81749505),
   'Sharpness': np.float32(8.596131),
   'WS': np.float32(2.75559)}},
 1: {'gnn': {'CQR': (np.float64(0.9106557377049179), np.float32(3.0860302))},
  'conf_gnn': {'CQR': (np.float64(0.9099453551912569), np.float32(3.6170883)),
   'eff_valid': np.float32(3.7218518)},
  'metrics': {'CWC': np.float64(1.5372805512371268),
   'PCIP': np.float64(0.8961748633879781),
   'NMPIW': np.float32(0.7539413),
   'MCT': np.float64(0.013701672241336104),
   'MPE': np.float32(1.3173305),
   'Sharpness': np.float32(13.106194),
   'WS': np.float32(3.61565)}},
 2: {'gnn': {'CQR': (np.float64(0.911803278688524

In [3]:
pd.read_pickle('./pred/Anaheim_GraphSAGE_confgnn.pkl')

{0: {'gnn': {'CQR': (np.float64(0.911967213114754), np.float32(2.5212917))},
  'conf_gnn': {'CQR': (np.float64(0.9098360655737704), np.float32(3.3122408)),
   'eff_valid': np.float32(3.1275687)},
  'train_metrics': {'CWC': np.float64(0.35186425487992434),
   'PCIP': np.float64(0.6182217774564398),
   'NMPIW': np.float32(0.34989938),
   'MCT': np.float64(0.9625668440597598),
   'MPE': np.float32(0.8438312),
   'Sharpness': np.float32(3.5468836),
   'WS': np.float32(2.0895555)},
  'test_metrics': {'CWC': np.float64(0.3927192267302672),
   'PCIP': np.float64(0.6455477918122369),
   'NMPIW': np.float32(0.39104834),
   'MCT': np.float64(1.0135666867358855),
   'MPE': np.float32(0.8090531),
   'Sharpness': np.float32(3.561245),
   'WS': np.float32(2.073971)}},
 1: {'gnn': {'CQR': (np.float64(0.9113114754098359), np.float32(2.8597443))},
  'conf_gnn': {'CQR': (np.float64(0.9113114754098359), np.float32(2.8597443)),
   'eff_valid': np.float32(2.7350974)},
  'train_metrics': {'CWC': np.float64(